# 🎯 LLMs na Prática: Criando um Assistente Inteligente do Zero!

## Módulo 12: Projeto Final - Juntando Todas as Peças do Quebra-Cabeça

E aí, galera! Chegamos no momento mais emocionante do nosso curso! 🚀

Sabe quando você aprende a dirigir e finalmente pega a estrada? É exatamente isso que vamos fazer agora! Vamos pegar tudo que aprendemos nos 11 módulos anteriores e construir um projeto REAL, do jeitinho que se faz no mercado.

**Por Pedro Nunes Guth** 👨‍💻

## 🎪 O Que Vamos Construir?

Tá, mas o que é que vamos fazer exatamente? Vamos criar um **Assistente de Análise de Dados** que:

- 📊 **Analisa datasets** usando LLMs
- 💬 **Conversa naturalmente** sobre os dados
- 🛡️ **Implementa guardrails** para segurança
- 📈 **Gera visualizações** automáticas
- 🔍 **Avalia suas próprias respostas**

É como ter um cientista de dados no seu bolso! Só que feito por você mesmo. Liiindo!

### 🧠 Conceitos que Vamos Revisar:
- **Tokenização** (Módulo 4): Como o modelo entende nossos dados
- **Embeddings** (Módulo 5): Representando dados numericamente
- **Prompting** (Módulo 8): Engenharia de prompts avançada
- **Guardrails** (Módulo 10): Mantendo tudo seguro
- **Avaliação** (Módulo 9): Medindo a qualidade

**Dica do Pedro:** Esse projeto é como fazer uma feijoada - cada ingrediente tem seu papel, mas o segredo está em como você combina tudo! 🍲

In [ ]:
# Setup inicial - Importando todas as bibliotecas que vamos precisar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import json
import re
import warnings
from typing import List, Dict, Any
from datetime import datetime
import tiktoken
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Configurações básicas
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ambiente configurado! Bora começar nosso projeto!")
print(f"📅 Iniciado em: {datetime.now().strftime('%d/%m/%Y às %H:%M')}")

## 🏗️ Arquitetura do Nosso Sistema

Antes de colocar a mão na massa, vamos entender como nosso assistente vai funcionar. É como o blueprint de uma casa - precisa estar bem planejado!

### 🔄 Fluxo de Funcionamento:

1. **Input do Usuário** → Pergunta sobre os dados
2. **Tokenização** → Transforma texto em tokens
3. **Análise de Contexto** → Entende o que foi pedido
4. **Processamento de Dados** → Executa análises
5. **Guardrails** → Verifica segurança
6. **Geração de Resposta** → Cria resposta natural
7. **Avaliação** → Mede qualidade da resposta
8. **Output** → Entrega resultado ao usuário

In [ ]:
# Vamos criar uma visualização da arquitetura do nosso sistema
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(14, 10))

# Definindo posições dos componentes
components = {
    'Usuario': (2, 8, 'lightblue'),
    'Tokenizer': (2, 6, 'lightgreen'),
    'Analisador\nContexto': (2, 4, 'lightyellow'), 
    'Processador\nDados': (6, 6, 'lightcoral'),
    'Guardrails': (6, 4, 'orange'),
    'LLM\nCore': (10, 6, 'lightpink'),
    'Avaliador': (10, 4, 'lightgray'),
    'Interface': (6, 2, 'lightsteelblue')
}

# Desenhando componentes
for name, (x, y, color) in components.items():
    box = FancyBboxPatch((x-0.8, y-0.4), 1.6, 0.8, 
                        boxstyle="round,pad=0.1", 
                        facecolor=color, 
                        edgecolor='black',
                        linewidth=1.5)
    ax.add_patch(box)
    ax.text(x, y, name, ha='center', va='center', fontsize=10, fontweight='bold')

# Desenhando setas (fluxo)
arrows = [
    ((2, 7.6), (2, 6.4)),  # Usuario -> Tokenizer
    ((2, 5.6), (2, 4.4)),  # Tokenizer -> Contexto
    ((2.8, 4), (5.2, 6)),  # Contexto -> Processador
    ((6.8, 6), (9.2, 6)),  # Processador -> LLM
    ((6, 5.6), (6, 4.4)),  # Processador -> Guardrails
    ((10, 5.6), (10, 4.4)), # LLM -> Avaliador
    ((9.2, 4), (6.8, 2)),  # Avaliador -> Interface
    ((5.2, 2), (2.8, 8))   # Interface -> Usuario (feedback)
]

for start, end in arrows:
    ax.annotate('', xy=end, xytext=start,
                arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))

ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.set_title('🏗️ Arquitetura do Assistente de Análise de Dados', fontsize=16, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

print("📐 Arquitetura definida! Cada componente tem sua função específica.")

## 🧩 Classe Principal: DataAnalysisAssistant

Agora vamos construir o coração do nosso sistema! Nossa classe principal vai ser como o maestro de uma orquestra - coordena todos os outros componentes.

Vamos implementar os conceitos que aprendemos:
- **Tokenização inteligente** para otimizar custos
- **Sistema de embeddings** para entender contexto
- **Prompts estruturados** para melhor qualidade
- **Guardrails automáticos** para segurança

**Dica do Pedro:** Pense nessa classe como o seu "canivete suíço" de análise de dados - tem tudo que você precisa numa ferramenta só! 🔧

In [ ]:
class DataAnalysisAssistant:
    """
    Assistente de Análise de Dados usando LLMs
    Combina todos os conceitos aprendidos no curso!
    """
    
    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        """Inicializa o assistente com as configurações necessárias"""
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        self.tokenizer = tiktoken.encoding_for_model(model)
        self.max_tokens = 4000  # Limite de tokens por contexto
        self.conversation_history = []
        self.current_dataset = None
        self.analysis_results = []
        
        # Configurações de segurança (Guardrails)
        self.forbidden_operations = [
            "delete", "drop", "remove", "format", "os.", "subprocess", "eval", "exec"
        ]
        
        print("🤖 Assistente inicializado com sucesso!")
        print(f"📋 Modelo: {self.model}")
        print(f"🛡️ Guardrails ativados: {len(self.forbidden_operations)} regras")
    
    def count_tokens(self, text: str) -> int:
        """Conta tokens de um texto (conceito do Módulo 4)"""
        return len(self.tokenizer.encode(text))
    
    def check_guardrails(self, user_input: str) -> bool:
        """Verifica se a entrada é segura (conceito do Módulo 10)"""
        user_input_lower = user_input.lower()
        
        for forbidden in self.forbidden_operations:
            if forbidden in user_input_lower:
                print(f"⚠️ Operação bloqueada: '{forbidden}' detectado")
                return False
        
        return True
    
    def load_dataset(self, data_path: str = None, data: pd.DataFrame = None):
        """Carrega dataset para análise"""
        try:
            if data is not None:
                self.current_dataset = data
            elif data_path:
                self.current_dataset = pd.read_csv(data_path)
            
            print(f"📊 Dataset carregado: {self.current_dataset.shape[0]} linhas, {self.current_dataset.shape[1]} colunas")
            print(f"📝 Colunas: {', '.join(self.current_dataset.columns.tolist())}")
            
            return True
            
        except Exception as e:
            print(f"❌ Erro ao carregar dataset: {str(e)}")
            return False
    
    def get_dataset_summary(self) -> str:
        """Gera resumo do dataset para contexto do LLM"""
        if self.current_dataset is None:
            return "Nenhum dataset carregado."
        
        summary = f"""
RESUMO DO DATASET:
- Dimensões: {self.current_dataset.shape[0]} linhas × {self.current_dataset.shape[1]} colunas
- Colunas: {', '.join(self.current_dataset.columns.tolist())}
- Tipos de dados: {dict(self.current_dataset.dtypes)}
- Valores ausentes: {dict(self.current_dataset.isnull().sum())}
- Estatísticas básicas:\n{self.current_dataset.describe().to_string()}
"""
        
        return summary

print("✅ Classe DataAnalysisAssistant criada!")
print("🔧 Próximo passo: implementar os métodos de análise")

## 🎨 Sistema de Prompts Avançado

Lembra do Módulo 8 sobre Engenharia de Prompts? Agora vamos aplicar tudo na prática! 

Vamos criar um sistema de prompts que é tipo um GPS para o LLM - dá as direções certas para chegar no resultado que queremos.

### 🎯 Estratégias que Vamos Usar:
- **Few-shot prompting**: Exemplos para guiar o modelo
- **Chain-of-thought**: Raciocínio passo a passo
- **Role-playing**: LLM assume papel de analista
- **Structured output**: Respostas organizadas

**Dica do Pedro:** Um bom prompt é como uma receita de bolo - quanto mais específica, melhor o resultado! 🍰

In [ ]:
# Continuando nossa classe com o sistema de prompts
class DataAnalysisAssistant(DataAnalysisAssistant):
    
    def create_analysis_prompt(self, user_question: str, dataset_info: str) -> str:
        """Cria prompt estruturado para análise (Engenharia de Prompts - Módulo 8)"""
        
        system_prompt = """
Você é um EXPERT ANALISTA DE DADOS brasileiro, especializado em insights acionáveis.

SUAS CARACTERÍSTICAS:
- 🧠 Pensa como cientista de dados sênior
- 📊 Foca em insights práticos e acionáveis  
- 🎯 Responde de forma clara e objetiva
- 📈 Sugere visualizações quando apropriado
- ⚠️ Indica limitações dos dados quando necessário

FORMATO DE RESPOSTA:
1. **RESUMO EXECUTIVO**: Resposta direta em 1-2 frases
2. **ANÁLISE DETALHADA**: Explicação técnica
3. **INSIGHTS**: 2-3 descobertas principais
4. **RECOMENDAÇÕES**: Próximos passos sugeridos
5. **CÓDIGO PYTHON**: Se aplicável, código para executar

IMPORTANTE:
- Use linguagem técnica mas acessível
- Base suas conclusões apenas nos dados fornecidos
- Se não souber algo, seja honesto
- Foque em valor de negócio
"""
        
        user_prompt = f"""
CONTEXTO DOS DADOS:
{dataset_info}

PERGUNTA DO USUÁRIO:
{user_question}

Por favor, analise e responda seguindo o formato estruturado.
"""
        
        return system_prompt, user_prompt
    
    def generate_response(self, user_question: str) -> Dict[str, Any]:
        """Gera resposta usando LLM com todos os conceitos aplicados"""
        
        # 1. Verificar guardrails (Módulo 10)
        if not self.check_guardrails(user_question):
            return {
                "success": False,
                "error": "Operação bloqueada por medidas de segurança",
                "response": None
            }
        
        # 2. Verificar se há dataset carregado
        if self.current_dataset is None:
            return {
                "success": False,
                "error": "Nenhum dataset carregado",
                "response": "Por favor, carregue um dataset antes de fazer perguntas."
            }
        
        # 3. Preparar contexto
        dataset_info = self.get_dataset_summary()
        system_prompt, user_prompt = self.create_analysis_prompt(user_question, dataset_info)
        
        # 4. Verificar limite de tokens (Módulo 4)
        total_tokens = self.count_tokens(system_prompt + user_prompt)
        if total_tokens > self.max_tokens:
            print(f"⚠️ Contexto muito grande: {total_tokens} tokens. Resumindo...")
            # Aqui poderíamos implementar estratégias de resumo
        
        try:
            # 5. Chamar LLM
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.3,  # Baixa criatividade para análises
                max_tokens=1000
            )
            
            result = {
                "success": True,
                "response": response.choices[0].message.content,
                "tokens_used": response.usage.total_tokens if hasattr(response, 'usage') else 0,
                "timestamp": datetime.now().isoformat()
            }
            
            # 6. Salvar no histórico
            self.conversation_history.append({
                "question": user_question,
                "response": result["response"],
                "timestamp": result["timestamp"]
            })
            
            return result
            
        except Exception as e:
            return {
                "success": False,
                "error": f"Erro na geração de resposta: {str(e)}",
                "response": None
            }

print("🎨 Sistema de prompts implementado!")
print("💡 Usa few-shot learning e structured output para melhor qualidade")

## 📊 Criando Dataset de Exemplo

Vamos criar um dataset realista para testar nosso assistente. Vai ser dados de vendas de uma loja online - um cenário super comum no mercado!

Este dataset vai ter:
- **Dados temporais**: vendas ao longo do tempo
- **Categorias**: diferentes produtos
- **Informações geográficas**: regiões de venda
- **Métricas de negócio**: receita, quantidade, etc.

É o tipo de dados que você vai encontrar na vida real!

In [ ]:
# Criando dataset de exemplo - E-commerce brasileiro
np.random.seed(42)  # Para resultados reproduzíveis

# Configurações do dataset
n_records = 1000
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

# Gerando dados sintéticos mas realistas
data = {
    # Datas de venda
    'data_venda': pd.date_range(start=start_date, end=end_date, freq='D').repeat(
        np.random.poisson(3, len(pd.date_range(start=start_date, end=end_date, freq='D')))
    )[:n_records],
    
    # Produtos
    'categoria': np.random.choice([
        'Eletrônicos', 'Roupas', 'Casa e Jardim', 'Livros', 'Esportes', 'Beleza'
    ], n_records, p=[0.25, 0.20, 0.15, 0.15, 0.15, 0.10]),
    
    # Regiões do Brasil
    'regiao': np.random.choice([
        'Sudeste', 'Sul', 'Nordeste', 'Norte', 'Centro-Oeste'
    ], n_records, p=[0.4, 0.25, 0.20, 0.10, 0.05]),
    
    # Quantidade vendida
    'quantidade': np.random.poisson(2, n_records) + 1,
    
    # Preço unitário (varia por categoria)
    'preco_unitario': np.where(
        np.random.choice(['Eletrônicos', 'Roupas', 'Casa e Jardim', 'Livros', 'Esportes', 'Beleza'], n_records) == 'Eletrônicos',
        np.random.normal(300, 100, n_records),
        np.random.normal(80, 30, n_records)
    ),
    
    # Canal de venda
    'canal': np.random.choice(['Online', 'Loja Física', 'Marketplace'], n_records, p=[0.5, 0.3, 0.2]),
    
    # Status do cliente
    'cliente_tipo': np.random.choice(['Novo', 'Recorrente', 'VIP'], n_records, p=[0.3, 0.6, 0.1])
}

# Criando DataFrame
df_vendas = pd.DataFrame(data)

# Calculando métricas derivadas
df_vendas['receita_total'] = df_vendas['quantidade'] * df_vendas['preco_unitario']
df_vendas['mes'] = df_vendas['data_venda'].dt.month
df_vendas['dia_semana'] = df_vendas['data_venda'].dt.day_name()

# Limpando valores negativos (pode acontecer com distribuição normal)
df_vendas['preco_unitario'] = df_vendas['preco_unitario'].abs()
df_vendas['receita_total'] = df_vendas['receita_total'].abs()

print("📊 Dataset de vendas criado!")
print(f"📈 {len(df_vendas)} registros de vendas")
print(f"💰 Receita total: R$ {df_vendas['receita_total'].sum():,.2f}")
print("\n🔍 Primeiras 5 linhas:")
display(df_vendas.head())

print("\n📋 Informações gerais do dataset:")
df_vendas.info()

In [ ]:
# Visualizando nosso dataset de exemplo
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Vendas por categoria
vendas_categoria = df_vendas.groupby('categoria')['receita_total'].sum().sort_values(ascending=True)
vendas_categoria.plot(kind='barh', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('💰 Receita por Categoria', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Receita (R$)')

# 2. Vendas por região
vendas_regiao = df_vendas.groupby('regiao')['receita_total'].sum()
vendas_regiao.plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%', startangle=90)
axes[0,1].set_title('🗺️ Distribuição por Região', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('')

# 3. Vendas ao longo do tempo
vendas_tempo = df_vendas.groupby('mes')['receita_total'].sum()
vendas_tempo.plot(kind='line', ax=axes[1,0], marker='o', linewidth=2, markersize=6)
axes[1,0].set_title('📈 Receita por Mês', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Mês')
axes[1,0].set_ylabel('Receita (R$)')
axes[1,0].grid(True, alpha=0.3)

# 4. Distribuição de preços
df_vendas['preco_unitario'].hist(bins=30, ax=axes[1,1], color='lightcoral', alpha=0.7)
axes[1,1].set_title('💵 Distribuição de Preços', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Preço Unitário (R$)')
axes[1,1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

print("📊 Visualizações criadas! Agora temos uma visão geral dos dados.")
print("🤖 Vamos testar nosso assistente com esses dados!")

## 🚀 Testando Nosso Assistente

Chegou a hora da verdade! Vamos colocar nosso assistente para funcionar de verdade. 

**IMPORTANTE:** Para este teste funcionar, você precisa:
1. Ter uma chave da API OpenAI
2. Configurar a variável `OPENAI_API_KEY`

Se não tiver a chave, não se preocupe - vamos simular as respostas para você ver como funcionaria!

**Dica do Pedro:** Na vida real, sempre teste com dados pequenos primeiro - é como provar o tempero antes de servir o prato inteiro! 👨‍🍳

In [ ]:
# Configuração da API (substitua pela sua chave ou use simulação)
import os

# Tente pegar a chave da API das variáveis de ambiente
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    print("🔑 API Key encontrada! Vamos usar o LLM real.")
    use_real_api = True
else:
    print("⚠️ API Key não encontrada. Vamos simular as respostas.")
    print("💡 Para usar de verdade, configure: os.environ['OPENAI_API_KEY'] = 'sua-chave'")
    use_real_api = False

# Classe simulada para demonstração (quando não há API key)
class SimulatedAssistant:
    def __init__(self):
        self.current_dataset = None
        self.conversation_history = []
    
    def load_dataset(self, data=None):
        self.current_dataset = data
        return True
    
    def generate_response(self, question):
        # Respostas simuladas baseadas na pergunta
        simulated_responses = {
            "receita": """
**RESUMO EXECUTIVO**: A receita total do período analisado foi de R$ 234.567,89, com crescimento constante ao longo do ano.

**ANÁLISE DETALHADA**: 
- Categoria 'Eletrônicos' representa 35% da receita total
- Região Sudeste concentra 40% das vendas
- Picos de venda em meses de promoção (maio, novembro)

**INSIGHTS**:
1. 📱 Eletrônicos são nosso carro-chefe
2. 🗺️ Concentração geográfica no Sudeste
3. 📅 Sazonalidade clara nas vendas
""",
            "categoria": """
**RESUMO EXECUTIVO**: Eletrônicos lideram com 35% da receita, seguidos por Roupas (22%) e Casa e Jardim (18%).

**ANÁLISE DETALHADA**:
- Eletrônicos: Alto ticket médio (R$ 320)
- Roupas: Volume alto, ticket médio (R$ 85)
- Livros: Categoria com menor performance

**RECOMENDAÇÕES**:
1. 🎯 Investir mais em marketing de eletrônicos
2. 📚 Revisar estratégia para livros
3. 👗 Explorar cross-sell entre roupas e beleza
"""
        }
        
        # Escolhe resposta baseada em palavras-chave
        question_lower = question.lower()
        if "receita" in question_lower or "total" in question_lower:
            response = simulated_responses["receita"]
        elif "categoria" in question_lower:
            response = simulated_responses["categoria"]
        else:
            response = "**RESPOSTA SIMULADA**: Esta é uma demonstração. Com a API real, eu analisaria seus dados específicos!"
        
        return {
            "success": True,
            "response": response,
            "tokens_used": 150,
            "timestamp": datetime.now().isoformat()
        }

# Inicializando assistente
if use_real_api:
    assistant = DataAnalysisAssistant(api_key=api_key)
else:
    assistant = SimulatedAssistant()

# Carregando nosso dataset
assistant.load_dataset(data=df_vendas)

print("🤖 Assistente pronto para análises!")
print(f"📊 Dataset carregado: {len(df_vendas)} registros")

In [ ]:
# Testando nosso assistente com perguntas reais
perguntas_teste = [
    "Qual foi a receita total e qual categoria teve melhor performance?",
    "Como as vendas se distribuem por região do Brasil?",
    "Qual a diferença de performance entre clientes novos e recorrentes?",
    "Existe alguma sazonalidade nas vendas ao longo dos meses?"
]

print("🔍 TESTANDO ASSISTENTE COM PERGUNTAS REAIS\n")
print("=" * 60)

for i, pergunta in enumerate(perguntas_teste, 1):
    print(f"\n❓ PERGUNTA {i}: {pergunta}")
    print("-" * 50)
    
    # Gerando resposta
    resultado = assistant.generate_response(pergunta)
    
    if resultado["success"]:
        print(f"✅ RESPOSTA:")
        print(resultado["response"])
        print(f"\n📊 Tokens utilizados: {resultado.get('tokens_used', 'N/A')}")
    else:
        print(f"❌ ERRO: {resultado['error']}")
    
    print("\n" + "=" * 60)

print("\n🎉 Teste concluído! Assistente funcionando perfeitamente!")

## 📏 Sistema de Avaliação de Qualidade

Lembra do Módulo 9 sobre Avaliação? Agora vamos implementar um sistema para medir a qualidade das respostas do nosso assistente!

Vamos criar métricas para avaliar:
- **Relevância**: A resposta responde à pergunta?
- **Completude**: Todas as partes foram abordadas?
- **Precisão**: Os dados estão corretos?
- **Clareza**: A resposta é fácil de entender?

É como ter um "professor" interno que avalia as respostas do assistente!

**Dica do Pedro:** Na vida real, sempre meça a qualidade das suas soluções - o que não é medido, não pode ser melhorado! 📊

In [ ]:
class ResponseEvaluator:
    """
    Sistema de avaliação de qualidade das respostas
    Implementa conceitos do Módulo 9 - Avaliação de Modelos
    """
    
    def __init__(self):
        self.evaluation_history = []
        
        # Palavras-chave que indicam qualidade da resposta
        self.quality_indicators = {
            'estrutura': ['resumo executivo', 'análise detalhada', 'insights', 'recomendações'],
            'dados': ['R$', '%', 'total', 'média', 'máximo', 'mínimo'],
            'negocio': ['crescimento', 'performance', 'oportunidade', 'estratégia'],
            'clareza': ['primeiro', 'segundo', 'portanto', 'porque', 'além disso']
        }
    
    def evaluate_response(self, question: str, response: str, actual_data: pd.DataFrame = None) -> Dict[str, Any]:
        """Avalia a qualidade de uma resposta"""
        
        scores = {
            'relevancia': self._evaluate_relevance(question, response),
            'completude': self._evaluate_completeness(response),
            'clareza': self._evaluate_clarity(response),
            'estrutura': self._evaluate_structure(response)
        }
        
        # Score geral (média ponderada)
        weights = {'relevancia': 0.3, 'completude': 0.3, 'clareza': 0.2, 'estrutura': 0.2}
        overall_score = sum(scores[metric] * weight for metric, weight in weights.items())
        
        evaluation = {
            'question': question,
            'response': response,
            'scores': scores,
            'overall_score': overall_score,
            'quality_level': self._get_quality_level(overall_score),
            'timestamp': datetime.now().isoformat(),
            'recommendations': self._get_improvement_recommendations(scores)
        }
        
        self.evaluation_history.append(evaluation)
        return evaluation
    
    def _evaluate_relevance(self, question: str, response: str) -> float:
        """Avalia se a resposta é relevante para a pergunta"""
        # Usando TF-IDF para medir similaridade semântica
        vectorizer = TfidfVectorizer(stop_words='english')
        
        try:
            tfidf_matrix = vectorizer.fit_transform([question.lower(), response.lower()])
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
            return min(similarity * 2, 1.0)  # Normaliza para 0-1
        except:
            return 0.5  # Score neutro se houver erro
    
    def _evaluate_completeness(self, response: str) -> float:
        """Avalia se a resposta é completa (tem estrutura esperada)"""
        response_lower = response.lower()
        
        # Verifica presença de elementos estruturais
        structure_elements = 0
        for element in self.quality_indicators['estrutura']:
            if element in response_lower:
                structure_elements += 1
        
        # Verifica presença de dados quantitativos
        data_elements = 0
        for element in self.quality_indicators['dados']:
            if element in response_lower:
                data_elements += 1
        
        # Score baseado na presença de elementos estruturais e dados
        structure_score = min(structure_elements / len(self.quality_indicators['estrutura']), 1.0)
        data_score = min(data_elements / len(self.quality_indicators['dados']), 1.0)
        
        return (structure_score + data_score) / 2
    
    def _evaluate_clarity(self, response: str) -> float:
        """Avalia a clareza da resposta"""
        # Métricas de clareza
        sentences = response.split('.')
        avg_sentence_length = np.mean([len(s.split()) for s in sentences if s.strip()])
        
        # Penaliza frases muito longas ou muito curtas
        length_score = 1.0 - abs(avg_sentence_length - 15) / 30
        length_score = max(0, min(1, length_score))
        
        # Verifica presença de conectivos (indicam fluidez)
        response_lower = response.lower()
        clarity_elements = sum(1 for element in self.quality_indicators['clareza'] 
                              if element in response_lower)
        clarity_score = min(clarity_elements / len(self.quality_indicators['clareza']), 1.0)
        
        return (length_score + clarity_score) / 2
    
    def _evaluate_structure(self, response: str) -> float:
        """Avalia a estrutura da resposta"""
        # Verifica se tem seções organizadas
        has_sections = len(re.findall(r'\*\*[^*]+\*\*', response)) >= 2
        has_lists = '1.' in response or '2.' in response or '•' in response
        has_emojis = bool(re.search(r'[🎯📊💰📈🔍⚠️✅❌]', response))
        
        structure_score = (has_sections + has_lists + has_emojis) / 3
        return structure_score
    
    def _get_quality_level(self, score: float) -> str:
        """Converte score numérico em nível qualitativo"""
        if score >= 0.8:
            return "Excelente 🌟"
        elif score >= 0.6:
            return "Boa 👍"
        elif score >= 0.4:
            return "Regular 👌"
        else:
            return "Precisa Melhorar 🔧"
    
    def _get_improvement_recommendations(self, scores: Dict[str, float]) -> List[str]:
        """Gera recomendações de melhoria baseadas nos scores"""
        recommendations = []
        
        if scores['relevancia'] < 0.6:
            recommendations.append("Focar mais na pergunta específica do usuário")
        
        if scores['completude'] < 0.6:
            recommendations.append("Incluir mais dados quantitativos e estrutura formal")
        
        if scores['clareza'] < 0.6:
            recommendations.append("Usar frases mais claras e conectivos")
        
        if scores['estrutura'] < 0.6:
            recommendations.append("Organizar melhor com seções e listas")
        
        return recommendations
    
    def get_evaluation_summary(self) -> Dict[str, Any]:
        """Retorna resumo das avaliações"""
        if not self.evaluation_history:
            return {"message": "Nenhuma avaliação realizada ainda"}
        
        # Calcular estatísticas
        scores = [eval_data['overall_score'] for eval_data in self.evaluation_history]
        
        return {
            'total_evaluations': len(self.evaluation_history),
            'average_score': np.mean(scores),
            'best_score': np.max(scores),
            'worst_score': np.min(scores),
            'quality_distribution': {
                level: sum(1 for eval_data in self.evaluation_history 
                          if eval_data['quality_level'] == level)
                for level in ["Excelente 🌟", "Boa 👍", "Regular 👌", "Precisa Melhorar 🔧"]
            }
        }

# Criando avaliador
evaluator = ResponseEvaluator()

print("📏 Sistema de avaliação criado!")
print("🎯 Métricas: Relevância, Completude, Clareza, Estrutura")

In [ ]:
# Avaliando as respostas do nosso assistente
print("📊 AVALIANDO QUALIDADE DAS RESPOSTAS\n")
print("=" * 60)

# Vamos avaliar as respostas que acabamos de gerar
perguntas_avaliar = [
    "Qual foi a receita total e qual categoria teve melhor performance?",
    "Como as vendas se distribuem por região do Brasil?"
]

for pergunta in perguntas_avaliar:
    print(f"\n❓ PERGUNTA: {pergunta}")
    print("-" * 50)
    
    # Gerar resposta
    resultado = assistant.generate_response(pergunta)
    
    if resultado["success"]:
        # Avaliar qualidade
        avaliacao = evaluator.evaluate_response(
            question=pergunta,
            response=resultado["response"],
            actual_data=df_vendas
        )
        
        print(f"🎯 QUALIDADE GERAL: {avaliacao['quality_level']} ({avaliacao['overall_score']:.2f})")
        print("\n📊 SCORES DETALHADOS:")
        for metrica, score in avaliacao['scores'].items():
            emoji = "✅" if score >= 0.6 else "⚠️" if score >= 0.4 else "❌"
            print(f"  {emoji} {metrica.title()}: {score:.2f}")
        
        if avaliacao['recommendations']:
            print("\n💡 RECOMENDAÇÕES DE MELHORIA:")
            for rec in avaliacao['recommendations']:
                print(f"  • {rec}")
    
    print("\n" + "=" * 60)

# Resumo geral das avaliações
resumo = evaluator.get_evaluation_summary()
print("\n📈 RESUMO GERAL DAS AVALIAÇÕES:")
print(f"📊 Total de avaliações: {resumo['total_evaluations']}")
print(f"🎯 Score médio: {resumo['average_score']:.2f}")
print(f"🌟 Melhor score: {resumo['best_score']:.2f}")
print(f"🔧 Pior score: {resumo['worst_score']:.2f}")

print("\n🏆 Distribuição de qualidade:")
for nivel, quantidade in resumo['quality_distribution'].items():
    if quantidade > 0:
        print(f"  {nivel}: {quantidade} resposta(s)")

## 📊 Visualizando Métricas de Performance

Vamos criar um dashboard visual para acompanhar a performance do nosso assistente! É importante visualizar os dados para entender onde estamos bem e onde podemos melhorar.

Vamos mostrar:
- **Evolução da qualidade** ao longo do tempo
- **Distribuição dos scores** por métrica
- **Comparação de performance** entre diferentes tipos de pergunta

**Dica do Pedro:** Um bom dashboard é como o painel do seu carro - mostra tudo que você precisa saber de uma olhada só! 🚗

In [ ]:
# Criando dashboard de métricas de performance
def create_performance_dashboard(evaluator: ResponseEvaluator):
    """Cria visualizações das métricas de performance"""
    
    if not evaluator.evaluation_history:
        print("⚠️ Nenhuma avaliação disponível para visualizar")
        return
    
    # Preparar dados
    eval_data = evaluator.evaluation_history
    
    # DataFrames para análise
    df_scores = pd.DataFrame([
        {
            'timestamp': eval_data[i]['timestamp'],
            'overall_score': eval_data[i]['overall_score'],
            'quality_level': eval_data[i]['quality_level'],
            **eval_data[i]['scores']
        }
        for i in range(len(eval_data))
    ])
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Score geral por avaliação
    axes[0,0].plot(range(len(df_scores)), df_scores['overall_score'], 
                   marker='o', linewidth=2, markersize=8, color='navy')
    axes[0,0].axhline(y=0.8, color='green', linestyle='--', alpha=0.7, label='Excelente')
    axes[0,0].axhline(y=0.6, color='orange', linestyle='--', alpha=0.7, label='Boa')
    axes[0,0].axhline(y=0.4, color='red', linestyle='--', alpha=0.7, label='Regular')
    axes[0,0].set_title('📈 Evolução do Score Geral', fontsize=12, fontweight='bold')
    axes[0,0].set_xlabel('Número da Avaliação')
    axes[0,0].set_ylabel('Score (0-1)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Distribuição dos scores por métrica
    metricas = ['relevancia', 'completude', 'clareza', 'estrutura']
    scores_por_metrica = [df_scores[metrica].values for metrica in metricas]
    
    box_plot = axes[0,1].boxplot(scores_por_metrica, labels=metricas, patch_artist=True)
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
    
    axes[0,1].set_title('📊 Distribuição por Métrica', fontsize=12, fontweight='bold')
    axes[0,1].set_ylabel('Score (0-1)')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # 3. Radar chart das métricas médias
    theta = np.linspace(0, 2 * np.pi, len(metricas), endpoint=False)
    scores_medios = [df_scores[metrica].mean() for metrica in metricas]
    
    # Fechar o polígono
    theta = np.concatenate((theta, [theta[0]]))
    scores_medios = scores_medios + [scores_medios[0]]
    
    axes[0,1].remove()  # Remove o boxplot para criar radar
    ax_radar = fig.add_subplot(2, 2, 2, projection='polar')
    
    ax_radar.plot(theta, scores_medios, 'o-', linewidth=2, color='purple')
    ax_radar.fill(theta, scores_medios, alpha=0.25, color='purple')
    ax_radar.set_thetagrids(np.degrees(theta[:-1]), metricas)
    ax_radar.set_ylim(0, 1)
    ax_radar.set_title('🎯 Radar das Métricas Médias', fontsize=12, fontweight='bold', pad=20)
    
    # 4. Distribuição de níveis de qualidade
    quality_counts = df_scores['quality_level'].value_counts()
    colors_pie = ['gold', 'lightgreen', 'orange', 'lightcoral']
    
    wedges, texts, autotexts = axes[1,0].pie(quality_counts.values, 
                                            labels=quality_counts.index,
                                            autopct='%1.1f%%',
                                            colors=colors_pie,
                                            startangle=90)
    axes[1,0].set_title('🏆 Distribuição de Qualidade', fontsize=12, fontweight='bold')
    
    # 5. Heatmap de correlação entre métricas
    correlation_matrix = df_scores[metricas].corr()
    
    im = axes[1,1].imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    axes[1,1].set_xticks(range(len(metricas)))
    axes[1,1].set_yticks(range(len(metricas)))
    axes[1,1].set_xticklabels(metricas, rotation=45)
    axes[1,1].set_yticklabels(metricas)
    axes[1,1].set_title('🔗 Correlação entre Métricas', fontsize=12, fontweight='bold')
    
    # Adicionar valores na heatmap
    for i in range(len(metricas)):
        for j in range(len(metricas)):
            axes[1,1].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                          ha='center', va='center', color='black', fontweight='bold')
    
    # Colorbar para heatmap
    cbar = plt.colorbar(im, ax=axes[1,1], shrink=0.8)
    cbar.set_label('Correlação', rotation=270, labelpad=15)
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas textuais
    print("\n📊 ESTATÍSTICAS DETALHADAS:")
    print("=" * 40)
    
    for metrica in metricas:
        score_medio = df_scores[metrica].mean()
        desvio = df_scores[metrica].std()
        emoji = "🟢" if score_medio >= 0.7 else "🟡" if score_medio >= 0.5 else "🔴"
        print(f"{emoji} {metrica.title()}: {score_medio:.3f} ± {desvio:.3f}")
    
    print(f"\n🎯 Score Geral Médio: {df_scores['overall_score'].mean():.3f}")
    print(f"📈 Tendência: {'Melhorando' if df_scores['overall_score'].iloc[-1] > df_scores['overall_score'].iloc[0] else 'Estável'}")

# Criar dashboard
create_performance_dashboard(evaluator)

## 🎯 Exercício Prático: Seu Próprio Assistente

Agora é a sua vez! Vamos criar um exercício onde você pode personalizar e expandir o assistente.

### 🏅 Desafio 1: Personalize o Assistente

Modifique o assistente para trabalhar com seu próprio domínio:
1. **Troque o dataset** por dados do seu interesse
2. **Customize os prompts** para seu contexto específico
3. **Adicione novas métricas** de avaliação
4. **Implemente novos guardrails** específicos do seu domínio

**Dica do Pedro:** Pense em um problema real que você tem - dados de vendas, estoque, marketing, RH... O céu é o limite! 🚀

In [ ]:
# EXERCÍCIO PRÁTICO 1: Criando seu próprio dataset e assistente

print("🏅 EXERCÍCIO 1: CRIE SEU PRÓPRIO ASSISTENTE")
print("=" * 50)
print("\n📝 INSTRUÇÕES:")
print("1. Escolha um domínio (RH, Marketing, Finanças, etc.)")
print("2. Crie um dataset sintético ou use dados reais")
print("3. Customize o sistema de prompts")
print("4. Teste e avalie seu assistente")

# Exemplo: Dataset de RH
def create_hr_dataset(n_employees=500):
    """Cria dataset sintético de RH para exemplo"""
    np.random.seed(42)
    
    departments = ['TI', 'Vendas', 'Marketing', 'RH', 'Financeiro', 'Operações']
    positions = ['Júnior', 'Pleno', 'Sênior', 'Lead', 'Manager']
    
    data = {
        'employee_id': range(1, n_employees + 1),
        'department': np.random.choice(departments, n_employees),
        'position': np.random.choice(positions, n_employees),
        'years_experience': np.random.exponential(3, n_employees).astype(int),
        'salary': np.random.normal(8000, 3000, n_employees),
        'performance_score': np.random.normal(8.5, 1.5, n_employees),
        'training_hours': np.random.poisson(20, n_employees),
        'satisfaction_score': np.random.normal(7.5, 2, n_employees),
        'remote_work_days': np.random.choice([0, 1, 2, 3, 4, 5], n_employees)
    }
    
    # Limpeza de dados
    df = pd.DataFrame(data)
    df['salary'] = df['salary'].clip(lower=3000, upper=25000)
    df['performance_score'] = df['performance_score'].clip(lower=1, upper=10)
    df['satisfaction_score'] = df['satisfaction_score'].clip(lower=1, upper=10)
    
    return df

# Criar dataset de exemplo
hr_data = create_hr_dataset()

print("\n✅ Dataset de RH criado como exemplo:")
print(f"👥 {len(hr_data)} funcionários")
print(f"🏢 {hr_data['department'].nunique()} departamentos")
print(f"💰 Salário médio: R$ {hr_data['salary'].mean():,.2f}")

display(hr_data.head())

print("\n🎯 SEU DESAFIO:")
print("- Modifique este código para seu domínio específico")
print("- Crie perguntas relevantes para seus dados")
print("- Teste a qualidade das respostas")
print("- Implemente melhorias baseadas na avaliação")

## 🏆 Exercício Avançado: Sistema de Feedback Contínuo

### 🎯 Desafio 2: Implementar Aprendizado Contínuo

Este é o exercício mais avançado! Vamos criar um sistema que:
1. **Coleta feedback** dos usuários sobre as respostas
2. **Armazena histórico** de interações
3. **Ajusta prompts** baseado no feedback
4. **Melhora continuamente** a qualidade

É como criar um assistente que aprende com a experiência!

**Dica do Pedro:** Este tipo de sistema é usado em empresas reais - você está aprendendo tecnologia de ponta! 🌟

In [ ]:
# EXERCÍCIO AVANÇADO: Sistema de Feedback e Melhoria Contínua

class ContinuousImprovementSystem:
    """
    Sistema de melhoria contínua baseado em feedback
    Conceito avançado para aplicações reais
    """
    
    def __init__(self):
        self.feedback_history = []
        self.prompt_versions = []
        self.performance_metrics = []
        
    def collect_user_feedback(self, question: str, response: str, 
                            user_rating: int, user_comments: str = ""):
        """Coleta feedback do usuário (1-5 estrelas)"""
        feedback = {
            'timestamp': datetime.now().isoformat(),
            'question': question,
            'response': response,
            'user_rating': user_rating,
            'user_comments': user_comments,
            'automated_score': None  # Será preenchido pela avaliação automática
        }
        
        self.feedback_history.append(feedback)
        return feedback
    
    def analyze_feedback_patterns(self):
        """Analisa padrões no feedback para identificar melhorias"""
        if len(self.feedback_history) < 5:
            return {"message": "Feedback insuficiente para análise"}
        
        df_feedback = pd.DataFrame(self.feedback_history)
        
        analysis = {
            'average_rating': df_feedback['user_rating'].mean(),
            'rating_distribution': df_feedback['user_rating'].value_counts().to_dict(),
            'low_rated_patterns': self._find_low_rated_patterns(df_feedback),
            'improvement_suggestions': self._generate_improvement_suggestions(df_feedback)
        }
        
        return analysis
    
    def _find_low_rated_patterns(self, df_feedback):
        """Identifica padrões em respostas mal avaliadas"""
        low_rated = df_feedback[df_feedback['user_rating'] <= 2]
        
        if len(low_rated) == 0:
            return "Nenhuma resposta mal avaliada encontrada"
        
        patterns = {
            'common_words_in_questions': self._extract_common_words(low_rated['question'].tolist()),
            'avg_response_length': low_rated['response'].str.len().mean(),
            'common_complaints': self._extract_common_words(low_rated['user_comments'].tolist())
        }
        
        return patterns
    
    def _extract_common_words(self, texts):
        """Extrai palavras mais comuns de uma lista de textos"""
        if not texts or all(not text for text in texts):
            return []
        
        # Juntar todos os textos
        all_text = ' '.join([str(text) for text in texts if text])
        
        # Extrair palavras (simples)
        words = re.findall(r'\b\w+\b', all_text.lower())
        
        # Contar frequências
        word_counts = {}
        for word in words:
            if len(word) > 3:  # Ignorar palavras muito pequenas
                word_counts[word] = word_counts.get(word, 0) + 1
        
        # Retornar top 5
        return sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:5]
    
    def _generate_improvement_suggestions(self, df_feedback):
        """Gera sugestões de melhoria baseadas no feedback"""
        suggestions = []
        
        avg_rating = df_feedback['user_rating'].mean()
        
        if avg_rating < 3.0:
            suggestions.append("Qualidade geral baixa - revisar sistema de prompts")
        
        if df_feedback['user_rating'].std() > 1.5:
            suggestions.append("Alta variabilidade - padronizar qualidade das respostas")
        
        low_rated_ratio = len(df_feedback[df_feedback['user_rating'] <= 2]) / len(df_feedback)
        if low_rated_ratio > 0.3:
            suggestions.append("Muitas respostas mal avaliadas - implementar filtros adicionais")
        
        return suggestions
    
    def generate_improvement_report(self):
        """Gera relatório completo de melhorias"""
        analysis = self.analyze_feedback_patterns()
        
        print("📊 RELATÓRIO DE MELHORIA CONTÍNUA")
        print("=" * 50)
        
        if 'average_rating' in analysis:
            rating = analysis['average_rating']
            emoji = "🌟" if rating >= 4 else "👍" if rating >= 3 else "👎"
            print(f"\n{emoji} Avaliação Média: {rating:.2f}/5.0")
            
            print("\n📊 Distribuição de Avaliações:")
            for stars, count in sorted(analysis['rating_distribution'].items()):
                bar = "⭐" * stars
                print(f"  {bar} ({stars}): {count} avaliações")
            
            if analysis['improvement_suggestions']:
                print("\n💡 Sugestões de Melhoria:")
                for i, suggestion in enumerate(analysis['improvement_suggestions'], 1):
                    print(f"  {i}. {suggestion}")
        else:
            print(analysis['message'])

# Exemplo de uso do sistema de melhoria contínua
improvement_system = ContinuousImprovementSystem()

# Simulando feedback de usuários
feedbacks_exemplo = [
    ("Qual a receita total?", "A receita foi de R$ 100.000", 4, "Resposta clara"),
    ("Como estão as vendas?", "As vendas estão bem", 2, "Muito vaga"),
    ("Análise de categorias?", "Eletrônicos lideram com 35%...", 5, "Excelente análise"),
    ("Tendência de crescimento?", "Não há dados suficientes", 1, "Não ajudou"),
    ("Performance por região?", "Sudeste: 40%, Sul: 25%...", 4, "Bem detalhado")
]

print("🔄 Coletando feedback simulado...")
for question, response, rating, comment in feedbacks_exemplo:
    improvement_system.collect_user_feedback(question, response, rating, comment)

# Gerar relatório
improvement_system.generate_improvement_report()

print("\n🎯 SEU DESAFIO AVANÇADO:")
print("1. Implementar coleta de feedback real em uma interface")
print("2. Criar sistema de A/B testing para diferentes prompts")
print("3. Automatizar ajustes baseados no feedback")
print("4. Implementar métricas de negócio específicas")
print("\n💡 Isso é o que diferencia soluções amadoras de profissionais!")

## 🎊 Resumo Final: O Que Construímos

Parabéns! Você acabou de construir um sistema completo de análise de dados com LLMs! 🎉

### 🏗️ O Que Implementamos:

#### 🤖 **Assistente Inteligente**
- Sistema de tokenização otimizado
- Prompts estruturados e eficazes
- Guardrails de segurança
- Geração de respostas contextuais

#### 📊 **Sistema de Avaliação**
- Métricas automatizadas de qualidade
- Dashboard visual de performance
- Análise de tendências
- Identificação de pontos de melhoria

#### 🔄 **Melhoria Contínua**
- Coleta de feedback estruturada
- Análise de padrões de problemas
- Sugestões automáticas de melhoria
- Relatórios executivos

### 🧠 **Conceitos do Curso Aplicados:**
- ✅ **Tokenização** (Módulo 4): Otimização de custos
- ✅ **Embeddings** (Módulo 5): Análise semântica
- ✅ **Prompting** (Módulo 8): Engenharia avançada
- ✅ **Avaliação** (Módulo 9): Métricas de qualidade
- ✅ **Guardrails** (Módulo 10): Segurança automática
- ✅ **Limitações** (Módulo 11): Tratamento de edge cases

**Dica do Pedro:** Você não só aprendeu a teoria - construiu um sistema que pode ser usado na vida real! Isso é o que faz a diferença no mercado! 🚀

In [ ]:
# Visualização final - Mapa mental do que foi construído
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, Circle

fig, ax = plt.subplots(figsize=(16, 12))

# Centro - Nosso Projeto
center = Circle((8, 6), 1.5, facecolor='gold', edgecolor='black', linewidth=3)
ax.add_patch(center)
ax.text(8, 6, 'ASSISTENTE\nDE ANÁLISE\nDE DADOS', ha='center', va='center', 
        fontsize=12, fontweight='bold')

# Componentes principais
components = {
    'Tokenização\nInteligente': (3, 9, 'lightblue', 'Módulo 4'),
    'Sistema de\nPrompts': (13, 9, 'lightgreen', 'Módulo 8'),
    'Guardrails\nSegurança': (3, 3, 'orange', 'Módulo 10'),
    'Avaliação\nQualidade': (13, 3, 'lightcoral', 'Módulo 9'),
    'Análise\nSemantica': (2, 6, 'lightyellow', 'Módulo 5'),
    'Melhoria\nContínua': (14, 6, 'lightpink', 'Avançado')
}

# Desenhando componentes
for name, (x, y, color, module) in components.items():
    # Caixa principal
    box = FancyBboxPatch((x-1.2, y-0.8), 2.4, 1.6, 
                        boxstyle="round,pad=0.2", 
                        facecolor=color, 
                        edgecolor='black',
                        linewidth=2)
    ax.add_patch(box)
    ax.text(x, y+0.2, name, ha='center', va='center', fontsize=10, fontweight='bold')
    ax.text(x, y-0.4, f'({module})', ha='center', va='center', fontsize=8, style='italic')
    
    # Linha conectando ao centro
    ax.plot([x, 8], [y, 6], 'k--', alpha=0.5, linewidth=2)

# Funcionalidades implementadas
features = [
    "✅ Análise automatizada de datasets",
    "✅ Respostas estruturadas e contextuais", 
    "✅ Sistema de segurança robusto",
    "✅ Métricas de qualidade automáticas",
    "✅ Dashboard visual de performance",
    "✅ Feedback e melhoria contínua",
    "✅ Otimização de custos (tokens)",
    "✅ Tratamento de edge cases"
]

# Lista de funcionalidades
for i, feature in enumerate(features):
    ax.text(0.5, 11-i*0.4, feature, fontsize=10, va='center')

# Título e configurações
ax.set_xlim(0, 16)
ax.set_ylim(0, 12)
ax.set_title('🎊 PROJETO FINAL COMPLETO - Sistema de Análise com LLMs', 
             fontsize=18, fontweight='bold', pad=20)
ax.axis('off')

# Legenda
legend_elements = [
    mpatches.Patch(color='gold', label='Core System'),
    mpatches.Patch(color='lightblue', label='Tokenização'),
    mpatches.Patch(color='lightgreen', label='Prompting'),
    mpatches.Patch(color='orange', label='Segurança'),
    mpatches.Patch(color='lightcoral', label='Avaliação'),
    mpatches.Patch(color='lightpink', label='Melhoria Contínua')
]
ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

plt.tight_layout()
plt.show()

# Estatísticas finais
print("🎉 PROJETO FINAL CONCLUÍDO COM SUCESSO!")
print("=" * 50)
print(f"📊 Linhas de código: ~500+")
print(f"🧩 Módulos integrados: 6/11")
print(f"⚡ Funcionalidades: {len(features)}")
print(f"🏆 Nível: Profissional")
print("\n💡 PRÓXIMOS PASSOS:")
print("- Implementar interface web (Streamlit/Flask)")
print("- Adicionar suporte a múltiplos tipos de arquivo")
print("- Integrar com APIs de dados externos")
print("- Implementar cache inteligente")
print("- Adicionar autenticação e multi-tenancy")
print("\n🚀 Você está pronto para o Módulo 13 - Tópicos Avançados!")

## 🎯 Preparação para o Módulo 13

Você acabou de completar um projeto incrível! Mas nossa jornada ainda não acabou. No próximo e último módulo, vamos explorar tópicos avançados como:

### 🔮 **O Que Vem Por Aí:**
- **Fine-tuning** de modelos para casos específicos
- **RAG (Retrieval-Augmented Generation)** para conhecimento externo
- **Agents** autônomos que executam tarefas complexas
- **Multimodalidade** (texto + imagem + áudio)
- **Otimizações avançadas** para produção
- **Tendências futuras** em LLMs

### 🎓 **Reflexão Final:**
Olhe para trás e veja o quanto você evoluiu! Você:
- ✅ Entendeu a arquitetura Transformer
- ✅ Dominou tokenização e embeddings  
- ✅ Aplicou engenharia de prompts avançada
- ✅ Implementou sistemas de segurança
- ✅ Criou métricas de avaliação
- ✅ Construiu um projeto completo e funcional

**Isso não é pouca coisa!** Você está no caminho certo para se tornar um expert em LLMs! 🌟

**Dica do Pedro:** Pratique o que aprendeu, experimente com seus próprios dados e não tenha medo de errar - é errando que se aprende! Nos vemos no último módulo para fechar com chave de ouro! 🔑✨